**Event Detection in Social Media Streams**

In [ ]:
import nltk
nltk.download('stopwords') # downloading the stopwords corpus

In [ ]:
!pip install google-api-python-client pandas scikit-learn nltk wordcloud matplotlib seaborn


In [ ]:
#-----------------------step 1: importing libraries---------------------------
import pandas as pd # for data handling
import numpy as np # for numerical
import re # for matching patterns, searching and replacing. can group the parts of a pattern.
import matplotlib.pyplot as plt  # for visualization
import seaborn as sns  # for better visualization
from wordcloud import WordCloud # if the frequesncy of a word is greater than it will be representated in greater size. more the frequency greater the size
from sklearn.feature_extraction.text import TfidfVectorizer # TfidfVectorizer handles tokenizaton, lowercasing, removing stopwords. alos coverts to arrays
from sklearn.cluster import KMeans # to form clusters or say clustering tweets into events
import nltk  # natural language processing. has many features some are: remove the prefix and suffix from sentences and still keep its meaning, categorize words under noun, adjectives, verb,etc, help identify the words such as if the word is a location, thing, person
from nltk.tokenize import word_tokenize # breaking sentences into words
from nltk.corpus import stopwords # remove the irrelevent words such as if, and, are, etc of different language
from collections import Counter # helps in counting the words in a
from scipy.cluster.hierarchy import linkage, fcluster # linkage is used to perform hierarchical clustering.it finds the distance between the d
from scipy.cluster.hierarchy import linkage,fcluster,dendrogram # linkage is used for getting the klinkage matrix that tell us about the clusters. linkage matrix consist of: list of clusters being murged, distance between clusters, how many new data points are now part of the new cluster after the merge. fcluster is used to make a get: like maximum number of clusters,  how clusters are formed and represented after linkage.
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import csv


In [ ]:
df=pd.read_csv('/content/GBcomments.csv', on_bad_lines='skip', quoting=csv.QUOTE_MINIMAL,engine='python')
print(df.head())
print(df.columns)
df.rename(columns={'comment_text':'Comment'}, inplace=True)
print(df.columns)

In [ ]:
nltk.download('stopwords') # downloading stopwords
nltk.download('punkt_tab') # punkt is used to convert text into sentances and than words
def preprocess_text(text):
  text=str(text).lower() # coverting all the text in the comment to lowercase
  text=re.sub(r"https?://\S+",'',text) # re.sub is used to search a patter. removing the texts. the r represents the the data is raw which means that \ are used as literal characters not the escape characters.http part matches the literal string and s? after that means that s is optional which helps to match with both http:// and https:// . now,  :// also matches the literal characters. than again, \s matches the any non-whitespace  characters. the + is with \s which gives \s+ means typically the rest of the url
  text=re.sub(r"\W", ' ', text) # removing the speacial characters. \W matches any character that is not a word. it includes uppercase, lowercase, digits, u=underscore. it does not involve spaces,punctuation marks, speacial characters, any other character that won't be aphanumeric or underscore. we provided a blank space in ' ' becuse when we remove the punchuations and replace them with a blank space than it helps from getting the text from being corelated to each other
  text=re.sub(r"\s+",' ', text) # removing the extra spaces
  words=word_tokenize(text) # tokenizing. this involves  splitting text into individual words.
  words=[ word for word in words if word not in stopwords.words('english')] #Remove stopwords. stopwords(is, in, and, etc) carry very little meaning.
  return' '.join(words)
df["Cleaned_Comment"]=df['Comment'].apply(preprocess_text) # calling the function to clean the Comments
print(df[["Comment", "Cleaned_Comment"]].head())

In [ ]:
# now we need to convert texts into numbers( TF-IDF vectorization)
vectorizer=TfidfVectorizer(max_features=1000) # we are converting the texts into the matrixes to find how imaportant, rare the word is. the matrix consists of the corresponding TF-IDF score
x= vectorizer.fit_transform(df['Cleaned_Comment']) # converting the text data into numarical values
print(x.shape)

In [ ]:
# we will be applying k mean clustering
number_of_clusters=5 # limiting the number of clusters i want
kmeans=KMeans(n_clusters=number_of_clusters, random_state=42)
df['KMeans_Cluster']=kmeans.fit_predict(x)
print(df[["Comment", "KMeans_Cluster"]].head())

In [ ]:
# we will be Visualize Clusters with Word Clouds. a word cloud is a visual representation of text data based on the word size and its frequencies
for cluster in range(number_of_clusters):
  cluster_comments= df[df['KMeans_Cluster']==cluster]['Cleaned_Comment'] # df['KMeans_Cluster]==cluster creates a series of True and False which checks if the each row in Cluster matches the cluster. df[df['Cluster']==cluster] will return new dataframe which will only contains the rows where the Cluster column matches particular cluster. [cleaned_Comment] with the others will contain only the cleaned comments that will belong to a particualr cluster
  text=' '.join(cluster_comments) # the comments that are similar will form a string
  wordcloud=WordCloud(width=800, height=400).generate(text) # .generate(text) is used as it takes the text data and than gives it to the wordcloud for the visual representation
  plt.imshow(wordcloud, interpolation='bilinear') # interpolation is used to define the color of th pixels. billinear uses closest 2X2 matrix. it first perfoms the linear and than other directions. this results the smoother image as comparision to the naerest neighbor interpolation
  plt.axis("off")
  plt.title(f"KMeans Cluster {cluster}")
  plt.show()
  # this helped us to see the most common words in each cluster

In [ ]:
# we gotta identify the event in the clusters
for cluster in range (number_of_clusters):
  cluster_comments=df[df['KMeans_Cluster']==cluster]['Cleaned_Comment']
  all_words=' '.join(cluster_comments).split() # taking the cluster_comments and and join them in a single string with space. from here the split splits the string at whitespaces, tabs, newlines and return a list of words
  common_words=Counter(all_words).most_common(10) # 10 is to get the most common 10 words
  print(f"Cluster {cluster} top words: {common_words}")

In [ ]:
# we will detect the event
event_keywords=["earthquake", "protest", "accident", "breaking", "viral", "trending"]
def detect_events(cluster_comments):
  for keyword in event_keywords:
    if keyword in cluster_comments:
      return True
  return False
df['is event']=df['Cleaned_Comment'].apply(detect_events)
print(df[df['is event'] == True]) # printing the detected event

In [ ]:
# to see hpow well the clustering is
from sklearn.metrics import silhouette_score # if the score is near 1 than clustering is good. if its near than it is bad
silhouette_avg= silhouette_score(x, df["KMeans_Cluster"])
print(f"silhouette score:{silhouette_avg}")

In [ ]:
# elbow mwthod ( for checking how many clusters should be made)
inertia=[]
K=range(1,11)
for k in K:
  kmeans=KMeans(n_clusters=k,random_state=42)
  kmeans.fit(x)
  inertia.append(kmeans.inertia_)
plt.plot(K, inertia, 'bx-')
plt.xlabel("number of clusters")
plt.ylabel("inertia")
plt.title("elbow method")
plt.show()

In [ ]:
from sklearn.metrics import davies_bouldin_score # if the score is less it means good
db_score=davies_bouldin_score(x.toarray(), df["KMeans_Cluster"])
print(f"davies bouldin index: {db_score}")

In [ ]:
# visual inspection to see how far the clusters are: (if the silhouette score is less yet the number of cluster needed is good than one of the reasons could be that distance between the clusters colud be less)
# PCA (principal component analysis) ot t-SNE (t-distribution stochastic neighbor embedding) is used to see how well- separated clusters are
from sklearn.decomposition import PCA
pca=PCA(n_components=2)
reduced_data=pca.fit_transform(x.toarray())
plt.scatter(reduced_data[:,0], reduced_data[:, 1], c=df["KMeans_Cluster"], cmap="viridis", marker="o")
plt.title("pca clusters")
plt.xlabel("pca 1")
plt.ylabel("pca 2")
plt.show()